In [ ]:
#####################################
# This creates figure
# PrecExc500.png and 
# in paper
#####################################
import sys
# import modules in other directories
sys.path.append('../Utils/')
sys.path.append('/glade/work/juliob/PyRegridding/Regridder/')

import esmfRegrid as erg

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import trax_util as trx
import ibtracs as IBT
import TRMM_util as trmm
import Precip_util as prc
import PlotUtil as Pu
import esmfRegrid as erg


import importlib

import dask

import time as TimeUtils

importlib.reload ( Pu )
importlib.reload ( prc )
importlib.reload ( trmm )



In [ ]:
##
etag = 'exceedance_xTS'

In [ ]:
if (etag == 'exceedance_o'):
    scrip_dir = '/glade/p/cesmdata/cseg/inputdata/share/scripgrids/'
    dst_scrip = scrip_dir + 'fv0.23x0.31_141008.nc'
    src_scrip = scrip_dir + 'ne120np4_pentagons_100310.nc'

    print( f"About to create ESMF regridding objects ")
    #ne120 ==> latlon 0.25 degree
    regrd, srcf, dstf = erg.Regrid(srcScrip = src_scrip , 
                                    srcType  = 'mesh'  ,
                                    dstScrip = dst_scrip ,
                                    dstType  = 'grid'   )

    DstFieldData = dstf.data
    nx,ny = np.shape( DstFieldData ) # Note ESMF objects are transposed 
    print(f"Size of precip grid {nx,ny}")

    Scr=xr.open_dataset( dst_scrip )
    latQd_yx = Scr.grid_center_lat.values.reshape( (768, 1152 ) )
    lonQd_yx = Scr.grid_center_lon.values.reshape( (768, 1152 ) )
    lonQd = lonQd_yx[0,:]
    latQd = latQd_yx[:,0]


In [ ]:
##
tic_overall = TimeUtils.perf_counter()
##
year0,year1 = 1998,2012
nyrs = 0
prec_grand_X = 0.
for Yr in np.arange( start=year0 , stop=year1+1 ):
    yA = str( Yr ).zfill(4)
    npSaveFile = '/glade/work/juliob/NumPySaves/'+etag+'/'+'TRMM-'+etag+'-'+yA+'.npz'
    SaveData = np.load( npSaveFile )
    prec_X = SaveData['prec_X']
    prec_grand_X = prec_grand_X + prec_X
    #print(f"File = {npSaveFile} ")
    nyrs=nyrs+1

lono = SaveData['lon']
lato = SaveData['lat']
    
TRMM_prec_grand_X = 3*prec_grand_X / nyrs
toc_overall = TimeUtils.perf_counter()
pTime = f"Overall time creating prec exceedance   {toc_overall - tic_overall:0.4f} seconds"
print(pTime)

In [ ]:
tic_overall = TimeUtils.perf_counter()
##
year0,year1 = 1998,2012
nyrs = 0
prec_grand_X = 0.

for e in np.arange(start=1,stop=4):
    BaseName = trx.pdfname(ens=e,justBaseName=True) 
    print(f"{BaseName}")
    for Yr in np.arange( start=year0 , stop=year1+1 ):
        yA = str( Yr ).zfill(4)
        npSaveFile = '/glade/work/juliob/NumPySaves/'+etag+'/'+BaseName+'-'+etag+'-'+yA+'.npz'
        #print(f"File = {npSaveFile} ")
        SaveData = np.load( npSaveFile )
        prec_X = SaveData['prec_X']
        prec_grand_X = prec_grand_X + prec_X
        nyrs=nyrs+1

lonc = SaveData['lon']
latc = SaveData['lat']
    
PD_prec_grand_X = 3*prec_grand_X / nyrs
toc_overall = TimeUtils.perf_counter()
pTime = f"Overall time creating prec exceedance   {toc_overall - tic_overall:0.4f} seconds"
print(pTime)

In [ ]:
tic_overall = TimeUtils.perf_counter()
##
for sstA in ['sst7','sst6','sst2','sst1']:
    year0,year1 = 2070,2099
    nyrs = 0
    prec_grand_X = 0.
    BaseName = trx.rcp85fname(sst=sstA,justBaseName=True) 
    for Yr in np.arange( start=year0 , stop=year1+1 ):
        yA = str( Yr ).zfill(4)
        npSaveFile = '/glade/work/juliob/NumPySaves/'+etag+'/'+BaseName+'-'+etag+'-'+yA+'.npz'
        #print(f"File = {npSaveFile} ")
        SaveData = np.load( npSaveFile )
        prec_X = SaveData['prec_X']
        prec_grand_X = prec_grand_X + prec_X
        #print(f"File = {npSaveFile} ")
        nyrs=nyrs+1

    lonc = SaveData['lon']
    latc = SaveData['lat']

    var_name = sstA + '_prec_grand_X'    
    globals()[var_name] = 3*prec_grand_X / nyrs
    print( f"Created {var_name}")

toc_overall = TimeUtils.perf_counter()
pTime = f"Overall time creating prec exceedance   {toc_overall - tic_overall:0.4f} seconds"
print(pTime)

In [ ]:
print(np.shape(sst7_prec_grand_X))

In [ ]:
# Precip exceedance threshholds
tholds = [50.,100.,300.,500.,1000.]

In [ ]:
##
import copy
if (etag == 'exceedance_o'):
    
    prcQd = copy.deepcopy(PD_prec_grand_X)
    PD_prec_grand_X = np.zeros( (5,ny,nx))
    for w in np.arange(5):
        PD_prec_grand_X[w,:,:] = erg.HorzRG(aSrc = prcQd[w,:] , 
                    regrd = regrd , 
                    srcField=srcf , 
                    dstField=dstf , 
                    srcGridkey='c' ,
                    dstGridkey='yx' ) 
    prcQd = copy.deepcopy(sst7_prec_grand_X)
    sst7_prec_grand_X = np.zeros( (5,ny,nx))
    for w in np.arange(5):
        sst7_prec_grand_X[w,:,:] = erg.HorzRG(aSrc = prcQd[w,:] , 
                    regrd = regrd , 
                    srcField=srcf , 
                    dstField=dstf , 
                    srcGridkey='c' ,
                    dstGridkey='yx' ) 

    lonc = lonQd
    latc = latQd


In [ ]:
##




lonW = -130
lonE = -10
latS = -5
latN = 45
cLat = (latN + latS) / 2
cLon = (lonW + lonE) / 2
res = '110m'



w=1
cmapN='gist_rainbow'
if (etag == 'exceedance_o'):
    preclv=np.linspace(.1,200.1,num=11) 
if (etag == 'exceedance_xTS'):
    preclv=np.array([.1,1,2,3,6,8,10,12,14,16,18,20] )
    preclv=np.array([.1,.5,1,2,3,4,5,6,8,9,10,12,14,16] )

cmap=Pu.step_cmap( levels=preclv , cmapN=cmapN )

#projMAP  = ccrs.PlateCarree() #ccrs.Robinson(central_longitude=0.)
#projData = ccrs.PlateCarree()
pco=0
projData = ccrs.PlateCarree()
projMAP = ccrs.PlateCarree(central_longitude=180.)

yshpanlab=.07
fig=plt.figure(figsize=(20,10))
n=0
#=======================
n=n+1
Axes = Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, PD_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM PD hrs>100 mm d$^{-1}$ 1998-2012" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'A)' , size=16)
#=======================
n=n+1
Axes=Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, sst6_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST6 hrs>100 mm d$^{-1}$ 1998-2012" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'B)' , size=16)

#=======================
n=n+1
Axes = Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes , projection=projMAP  )
co = ax1.contourf( lonc, latc, sst7_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap ,extend='max', transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
#plt.colorbar( co , ax=ax1 )
crax=fig.add_axes( Pu.right_cbar_axes( Axes, xshift=0.05, yshift=.1,ysize=0.15) )
cbar=fig.colorbar( co, cax=crax, ax=ax1 )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST7 hrs>100 mm d$^{-1}$ 2070-2099" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'C)' , size=16)


#==================================================
#==================================================
if (etag == 'exceedance_o'):
    preclv=np.linspace(.1,10.1,num=6) 
    preclv=np.array( [0.1,1.,2.,3.,4.,6.,8.,10] )
if (etag == 'exceedance_xTS'):
    preclv=np.linspace(.1,5.1,num=6) 
    preclv=np.array( [0.1,.5,1.,2.,3.,4.,5.] )
    
cmap=Pu.step_cmap( levels=preclv , cmapN=cmapN )

w=3

#=======================
n=n+1
Axes = Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, PD_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM PD hrs>500 mm d$^{-1}$ 1998-2012" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'D)' , size=16)

#=======================
n=n+1
Axes=Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, sst6_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST6 ens. mean hrs>500 mm d$^{-1}$ 1998-2012")
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'E)' , size=16)

#=======================
n=n+1
Axes = Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes , projection=projMAP  )
co = ax1.contourf( lonc, latc, sst7_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap ,extend='max', transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
#plt.colorbar( co , ax=ax1 )
crax=fig.add_axes( Pu.right_cbar_axes( Axes, xshift=0.05, yshift=.1,ysize=0.15) )
cbar=fig.colorbar( co, cax=crax, ax=ax1 )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST7 hrs>500 mm d$^{-1}$ 2070-2099")
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab , 'F)' , size=16)



In [ ]:
##




lonW = -130
lonE = -10
latS = -5
latN = 45
cLat = (latN + latS) / 2
cLon = (lonW + lonE) / 2
res = '110m'



w=1
cmapN='gist_rainbow'
if (etag == 'exceedance_o'):
    preclv=np.linspace(.1,200.1,num=11) 
if (etag == 'exceedance_xTS'):
    preclv=np.array([.1,1,2,3,6,8,10,12,14,16,18,20] )
    preclv=np.array([.1,.5,1,2,3,4,5,6,8,9,10,12,14,16] )

cmap=Pu.step_cmap( levels=preclv , cmapN=cmapN )

#projMAP  = ccrs.PlateCarree() #ccrs.Robinson(central_longitude=0.)
#projData = ccrs.PlateCarree()
pco=0
projData = ccrs.PlateCarree()
projMAP = ccrs.PlateCarree(central_longitude=180.)

yshpanlab=.07
fig=plt.figure(figsize=(20,10))
n=0
#=======================
n=n+1
Axes = Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, PD_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM PD hrs>100 mm d$^{-1}$ 1998-2012" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'A)' , size=16)
#=======================
n=n+1
Axes=Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, sst1_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST1 hrs>100 mm d$^{-1}$ 1998-2012" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'B)' , size=16)

#=======================
n=n+1
Axes=Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, sst2_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST2 hrs>100 mm d$^{-1}$ 1998-2012" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'B)' , size=16)

#=======================
n=n+1
Axes=Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, sst6_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST6 hrs>100 mm d$^{-1}$ 1998-2012" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'B)' , size=16)

#=======================
n=n+1
Axes = Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes , projection=projMAP  )
co = ax1.contourf( lonc, latc, sst7_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap ,extend='max', transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
#plt.colorbar( co , ax=ax1 )
crax=fig.add_axes( Pu.right_cbar_axes( Axes, xshift=0.05, yshift=.1,ysize=0.15) )
cbar=fig.colorbar( co, cax=crax, ax=ax1 )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST7 hrs>100 mm d$^{-1}$ 2070-2099" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'C)' , size=16)



In [ ]:
##


sstA_prec_grand_X = ( sst1_prec_grand_X + sst2_prec_grand_X + sst6_prec_grand_X + sst7_prec_grand_X)/4.


lonW = -130
lonE = -10
latS = -5
latN = 45
cLat = (latN + latS) / 2
cLon = (lonW + lonE) / 2
res = '110m'

w=3
if (etag == 'exceedance_o'):
    preclv=np.array( [0.1,1.,2.,3.,4.,6.,8.,10] )
if (etag == 'exceedance_xTS'):
    preclv=np.array( [0.1,.5,1.,2.,3.,4.,5.] )
    preclv=np.array( [0.1,.2,.5,1.,2.,3.,4.] )
    preclv=np.array( [0.1,.2,.3,.5,1.,2.,3.] )
    
cmap=Pu.step_cmap( levels=preclv , cmapN=cmapN )

#projMAP  = ccrs.PlateCarree() #ccrs.Robinson(central_longitude=0.)
#projData = ccrs.PlateCarree()
pco=0
projData = ccrs.PlateCarree()
projMAP = ccrs.PlateCarree(central_longitude=180.)

yshpanlab=.07
fig=plt.figure(figsize=(20,10))
n=0
#=======================
n=n+1
Axes = Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, PD_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM PD hrs>500 mm d$^{-1}$ 1998-2012" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'A)' , size=16)
#=======================
n=n+1
Axes=Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, sst1_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST1 hrs>500 mm d$^{-1}$ 1998-2012" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'B)' , size=16)

#=======================
n=n+1
Axes=Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, sst2_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST2 hrs>500 mm d$^{-1}$ 1998-2012" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'C)' , size=16)

#=======================
n=n+1
Axes=Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, sst6_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST6 hrs>500 mm d$^{-1}$ 1998-2012" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'D)' , size=16)

#=======================
n=n+1
Axes = Pu.axes_def(n=n,nxplo=3,nyplo=2,ycor=0.75 )
ax1 = fig.add_axes( Axes , projection=projMAP  )
co = ax1.contourf( lonc, latc, sst7_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap ,extend='max', transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
#plt.colorbar( co , ax=ax1 )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST7 hrs>500 mm d$^{-1}$ 2070-2099" )
fig.text( Axes[0],Axes[1]+Axes[3]-yshpanlab, 'E)' , size=16)

crax=fig.add_axes( Pu.right_cbar_axes( Axes, xshift=0.05, yshift=.1,ysize=0.2) )
cbar=fig.colorbar( co, cax=crax, ax=ax1 )
BarLabel = "Hours year$^{-1}$ (0.27$^{\circ}$x0.35$^{\circ}$) $^{-1}$"
cbar.set_label( label=BarLabel,size=12)

"""
BarLabel = "Hours year$^{-1}$ (5$^{\circ}$)$^{-2}$"
cax = plt.axes([0.8, 0.84, 0.01, 0.2])
cbar = plt.colorbar(po,cax=cax,orientation='vertical')
cbar.set_label( label=BarLabel,size=16)
cbar.ax.tick_params(labelsize=12)
"""

Pu.save_link(png_filename = "PrecExc500.png")


In [ ]:
##


#sstA_prec_grand_X = ( sst1_prec_grand_X + sst2_prec_grand_X + sst6_prec_grand_X + sst7_prec_grand_X)/4.


lonW = -130
lonE = -10
latS = -5
latN = 45
cLat = (latN + latS) / 2
cLon = (lonW + lonE) / 2
res = '110m'

w=3
if (etag == 'exceedance_o'):
    preclv=np.array( [0.1,1.,2.,3.,4.,6.,8.,10] )
if (etag == 'exceedance_xTS'):
    preclv=np.array( [0.1,.5,1.,2.,3.,4.,5.] )
    preclv=np.array( [0.1,.2,.5,1.,2.,3.,4.] )
    preclv=np.array( [0.1,.2,.3,.5,1.,2.,3.] )
    
cmap=Pu.step_cmap( levels=preclv , cmapN=cmapN )

#projMAP  = ccrs.PlateCarree() #ccrs.Robinson(central_longitude=0.)
#projData = ccrs.PlateCarree()
pco=0
projData = ccrs.PlateCarree()
projMAP = ccrs.PlateCarree(central_longitude=180.)

yshpanlab=.07
fig=plt.figure(figsize=(20,10))
n,ipa=0,-1
#=======================
n=n+1
Axes = Pu.axes_def(n=n,nxplo=3,nyplo=1,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, PD_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM PD hrs>500 mm d$^{-1}$ 1998-2012" )

ipa=ipa+1
ax1.text(-0.08, 1.1, f"{chr(97 +ipa)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')
#=======================

n=n+1
Axes=Pu.axes_def(n=n,nxplo=3,nyplo=1,ycor=0.75 )
ax1 = fig.add_axes( Axes  , projection=projMAP )
co = ax1.contourf( lonc, latc, sst1_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap, extend='max',transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST1 hrs>500 mm d$^{-1}$ 1998-2012" )

ipa=ipa+1
ax1.text(-0.08, 1.1, f"{chr(97 +ipa)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')
#=======================
n=n+1
Axes = Pu.axes_def(n=n,nxplo=3,nyplo=1,ycor=0.75 )
ax1 = fig.add_axes( Axes , projection=projMAP  )
co = ax1.contourf( lonc, latc, sst7_prec_grand_X[w,:,:] ,levels=preclv,cmap=cmap ,extend='max', transform=projData )
ax1.set_global()
#ax1.set_extent([lonW, lonE, latS, latN], crs=projData )
#plt.colorbar( co , ax=ax1 )
ax1.coastlines(resolution='110m',color='black',linewidth=1)
ax1.set_title("CAM SST7 hrs>500 mm d$^{-1}$ 2070-2099" )

ipa=ipa+1
ax1.text(-0.08, 1.1, f"{chr(97 +ipa)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')



#======== Color Bar ========================
crax=fig.add_axes( Pu.under_cbar_axes( [0.25,0.45] , xsize=0.4 ) )
cbar=fig.colorbar( co, cax=crax, ax=ax1,orientation='horizontal' )
BarLabel = "Hours year$^{-1}$ (0.27$^{\circ}$x0.35$^{\circ}$) $^{-1}$"
cbar.set_label( label=BarLabel,size=12)

"""
BarLabel = "Hours year$^{-1}$ (5$^{\circ}$)$^{-2}$"
cax = plt.axes([0.8, 0.84, 0.01, 0.2])
cbar = plt.colorbar(po,cax=cax,orientation='vertical')
cbar.set_label( label=BarLabel,size=16)
cbar.ax.tick_params(labelsize=12)
"""

Pu.save_link(png_filename = "PrecExc500.png")


In [ ]:
print(Axes)

In [ ]:
###


# STOPO



####